In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import os

In [5]:
urls = {
    'Anxiety': 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}',
    'AnxietyMonths' : 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
}

In [6]:
def is_leap(year):
    if year % 4 != 0:
        return False
    elif year % 100 != 0:
        return True
    elif year % 400 != 0:
        return False
    else:
        return True
    
def convert_day(day, year):
    month_days = [31, 29 if is_leap(year) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [72]:
year = 2012
selected_days = [i for i in range(1, 367 if is_leap(year) else 366)]

In [73]:
img_dir = 'images'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)

In [75]:
data = []
article_id = 0
i = 0
n = len(selected_days)
if int(year) <= 2015:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
    for month in range(1,13):
        date = '{0}-{1:02d}'.format(year, month)
        print(url.format(year, month))
        response = requests.get(url.format(year, month), allow_redirects=True)
        if response.url.startswith(url.format(year, month)):
            page = response.content
            soup = BeautifulSoup(page, 'html.parser')
            articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
            for article in articles:
                title = article.find("h3", class_="graf--title")
                if title is None:
                    continue
                title = title.contents[0]
                article_id += 1
                article_url = article.find_all("a")[3]['href'].split('?')[0]

                data.append([article_id, article_url, title, date])
else:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}'
    for d in selected_days:
        i += 1
        month, day = convert_day(d, year)
        date = '{0}-{1:02d}-{2:02d}'.format(year, month, day)
        print(url.format(year, month, day))
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        for article in articles:
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            title = BeautifulSoup(title.contents[0])
            only_title = title.get_text()
            article_id += 1
            article_url = article.find_all("a")[3]['href'].split('?')[0]

            data.append([article_id, article_url, only_title, date])

            
            
            
            

https://medium.com/tag/anxiety/archive/2012/01
https://medium.com/tag/anxiety/archive/2012/02
https://medium.com/tag/anxiety/archive/2012/03
https://medium.com/tag/anxiety/archive/2012/04
https://medium.com/tag/anxiety/archive/2012/05
https://medium.com/tag/anxiety/archive/2012/06
https://medium.com/tag/anxiety/archive/2012/07
https://medium.com/tag/anxiety/archive/2012/08
https://medium.com/tag/anxiety/archive/2012/09
https://medium.com/tag/anxiety/archive/2012/10
https://medium.com/tag/anxiety/archive/2012/11
https://medium.com/tag/anxiety/archive/2012/12


In [84]:
# medium_df = pd.DataFrame(data, columns=['id', 'url', 'title', 'date'])

In [85]:
# medium_df

,id,url,title,date
0,1,https://medium.com/@parthsharma1996/a-simple-s...,"A simple, short technique to beat Anxiety",2017-01-01
1,2,https://medium.com/@inthefade/other-life-ff0d4...,other life,2017-01-01
2,3,https://medium.com/january-rain/another-eve-50...,Another Eve,2017-01-01
3,4,https://medium.com/@tommctighe/self-care-for-a...,Self-care for anxiety and panic,2017-01-01
4,5,https://medium.com/@pinealeyeopens/impromptu-e...,Impromptu,2017-01-01
...,...,...,...,...
5248,5249,https://medium.com/@epetpeeves/i-have-always-k...,"I Have Always Known I Am A Writer, Even At My ...",2017-12-31
5249,5250,https://medium.com/@julemo/anxiety-at-christma...,Anxiety At Christmas,2017-12-31
5250,5251,https://medium.com/@russelctgbd/11-healthy-way...,11 healthy ways to cope with stress,2017-12-31
5251,5252,https://medium.com/@puccaso/selfitis-behaviour...,Selfitis behaviour,2017-12-31


In [86]:
# medium_df.to_csv('medium_data_2017.csv', index=False)

In [76]:
for i in range(len(data)):
    url = data[i][1]
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    sections = soup.find_all('section')
    all_paragraphs = []
    section_titles = []
    
    for section in sections:
        paragraphs = section.find_all('p')
        for paragraph in paragraphs:
            all_paragraphs.append(paragraph.text)

        subs = section.find_all('h1')
        for sub in subs:
            only_text = BeautifulSoup(sub.text)
            section_titles.append(only_text.get_text())
    
    # to ensure that we only append to the data once (while testing)
    if len(data[i]) <= 4:
        sep = " "
        final_paragraphs = sep.join(all_paragraphs[2:])
        final_titles = sep.join(section_titles)
        data[i].append(final_titles + " " + final_paragraphs)
    
    print(data[i])

https://medium.com/@recoveryfromwithin/black-dog-and-the-light-at-the-end-of-the-tunnel-672ae61d8dab
[1, 'https://medium.com/@recoveryfromwithin/black-dog-and-the-light-at-the-end-of-the-tunnel-672ae61d8dab', 'Black Dog and the Light at the End of the\xa0Tunnel.', '2012-02', 'Black Dog and the Light at the End of the Tunnel. I wrote this blog in 2012 and it first appeared on my website The Midlife Coach. I stand by much of what I wrote back then, I’m grateful that the stigma is easily and that mental health is much more easily spoken about. We still have a way to go to get parity of understanding with physical health. 6 years later I wrote a follow up blog on what I was seeing differently. You can find it here. I’ve been shying away from this post for some time now. You know I don’t avoid tough subjects. Depression is one of those subjects. I am well and have been for many years now, that however, wasn’t always the case. My intention in writing about depression is to share my experienc

[4, 'https://sahilparikh.com/dealing-with-anxiety-2c1c4d6923b5', 'Dealing With\xa0Anxiety', '2012-08', ' ']
https://medium.com/plouffes-new-hairdo/feels-like-ages-staypositive-ef40bfe9f67e
[5, 'https://medium.com/plouffes-new-hairdo/feels-like-ages-staypositive-ef40bfe9f67e', 'Feels Like Ages… #StayPositive', '2012-08', 'Feels Like Ages… #StayPositive Well, I feel like I am rambling at this point. I have nothing to offer but speculation and hope. I am trying to remain positive, because what’s the point of being negative? If the Twins are your team, go ahead and show it by sticking with them through good times and bad. My childhood memories were made on “bad baseball”, I went to Twins games at the METRODOME when there were CONTRACTION talks, so don’t try to tell me that this is the worst. We have more to be thankful for, and maybe, just maybe, the positive vibes in the stands will flow over the limestone walls and the players will use it like a wave to propel them to new heights. Well, 

[10, 'https://medium.com/the-hairpin/assumptions-69126b72d845', 'Assumptions', '2012-11', 'Assumptions A hairball hack from my old cat is really the beginning stages of a death rattle. Any ache or pain will suddenly escalate, and then become permanent. If I let someone leave an argument before the problem is resolved, they will suddenly die and I will be stuck with the guilt of a bad parting. Every elderly person who lives alone is essentially Eleanor Rigby. That a weird garbage bag on the side of the road contains either the whole body of a child or assorted body parts of an adult. A phone call from my mother at an usual hour is to announce the death of a close relative. Any phone call from a close relative is to announce the death of my mother. That bug floating in my drink? It’s the widow of the bug I probably just drank. A person in my life knows all about that embarrassing message board I used to frequent and is just waiting to bring it up in front of a large group of people. I’m 

In [77]:
medium_df = pd.DataFrame(data, columns=['id', 'url', 'title', 'date', 'text'])

In [78]:
medium_df

,id,url,title,date,text
0,1,https://medium.com/@recoveryfromwithin/black-d...,Black Dog and the Light at the End of the Tunnel.,2012-02,Black Dog and the Light at the End of the Tunn...
1,2,https://medium.com/@IsMalkyDead/being-flat-in-...,Being Flat in the Land of Ups and Downs.,2012-06,Being Flat in the Land of Ups and Downs. I hav...
2,3,https://medium.com/@guiafon/emotional-and-phys...,Emotional And Physical Anxiety Why Does It Occur?,2012-07,Emotional And Physical Anxiety Why Does It Occ...
3,4,https://sahilparikh.com/dealing-with-anxiety-2...,Dealing With Anxiety,2012-08,
4,5,https://medium.com/plouffes-new-hairdo/feels-l...,Feels Like Ages… #StayPositive,2012-08,"Feels Like Ages… #StayPositive Well, I feel li..."
5,6,https://medium.com/plouffes-new-hairdo/still-a...,Still Anxiously Waiting…,2012-08,Still Anxiously Waiting… I am writing about my...
6,7,https://amysmartgirls.com/ask-amy-anxiety-9d39...,ASK AMY: ANXIETY,2012-09,ASK AMY: ANXIETY
7,8,https://medium.com/@JenniferLesher/this-is-how...,This is How I got over Writers Anxiety,2012-09,This is How I got over Writers Anxiety Much as...
8,9,https://medium.com/@stronginsideout/things-wil...,Things Will Work Out: Keeping The Faith in Tou...,2012-10,Things Will Work Out: Keeping The Faith in Tou...
9,10,https://medium.com/the-hairpin/assumptions-691...,Assumptions,2012-11,Assumptions A hairball hack from my old cat is...


In [79]:
medium_df = medium_df.drop(medium_df.columns[[0, 1, 3]], axis=1)

In [80]:
medium_df['year'] = 2012
medium_df

,title,text,year
0,Black Dog and the Light at the End of the Tunnel.,Black Dog and the Light at the End of the Tunn...,2012
1,Being Flat in the Land of Ups and Downs.,Being Flat in the Land of Ups and Downs. I hav...,2012
2,Emotional And Physical Anxiety Why Does It Occur?,Emotional And Physical Anxiety Why Does It Occ...,2012
3,Dealing With Anxiety,,2012
4,Feels Like Ages… #StayPositive,"Feels Like Ages… #StayPositive Well, I feel li...",2012
5,Still Anxiously Waiting…,Still Anxiously Waiting… I am writing about my...,2012
6,ASK AMY: ANXIETY,ASK AMY: ANXIETY,2012
7,This is How I got over Writers Anxiety,This is How I got over Writers Anxiety Much as...,2012
8,Things Will Work Out: Keeping The Faith in Tou...,Things Will Work Out: Keeping The Faith in Tou...,2012
9,Assumptions,Assumptions A hairball hack from my old cat is...,2012


In [81]:
medium_df.to_csv('medium_text_2012.csv', index=False)